In [1]:
import pandas as pd
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
# %matplotlib inline

path_data = "../data/"
path_fig = "../results/fig/"

In [2]:
# The following have been removed 
df_2019 = pd.read_pickle(path_data + "CPS_df_occmobpanel_2019.pkl")

In [2]:
df = pd.read_pickle(path_data + "CPS_df_occmob_summary.pkl")

In [3]:
df.head()

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,LNKFW1MWT,PANLWT
15,2000,1,233.3782,19981204103001,12,869,6260,60,242.8308,322.3957
20,2000,1,233.4906,19981204102801,12,783,8140,41,239.6893,320.5128
28,2000,1,290.0727,19981103262701,10,783,8140,41,298.4937,395.7209
34,2000,1,3309.0053,19981202764601,21,558,6200,60,3504.5631,4804.7770
36,2000,1,3285.7963,19981204464201,10,783,8140,282,3423.1920,4762.3957


In [4]:
len(df)

16869617

In [5]:
df['Idx'] = df.index
df = df.sort_values(by = ['CPSIDP', 'Idx'])

In [6]:
# NOTE there are some (were) NAN values in WTFINL, but there seems to be a previous entry in this case too.
# df[102700:102750]

In [6]:
df.head()

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,LNKFW1MWT,PANLWT,Idx
80373,2000,1,1929.3937,19981100003701,10,558,6200,60,1969.5642,2647.8748,80373
237164,2000,2,1957.8713,19981100003701,10,558,6200,60,0.0000,2651.8827,237164
80374,2000,1,1973.6605,19981100003702,10,323,5420,60,2011.3778,2693.7112,80374
237165,2000,2,2011.7397,19981100003702,10,323,5420,60,0.0000,2716.4674,237165
80375,2000,1,2004.5404,19981100003703,10,458,4510,772,2056.4402,2884.8375,80375


In [7]:
df_2019 = df[df["YEAR"] == 2019]
df_2019 = df_2019.sort_values(by = ['CPSIDP', 'Idx'])

In [33]:
# -1 to since we at least should have two observations
id_freq = (df_2019["CPSIDP"].value_counts() - 1).to_dict()

In [34]:
df_2019["counts"] = df_2019["CPSIDP"].map(id_freq)

In [35]:
df_2019["PANLWT_adj"] = df_2019["PANLWT"]/df_2019["counts"] 

In [36]:
df_2019_janfeb = df_2019[(df_2019["MONTH"] == 1) | (df_2019["MONTH"] == 2)]

In [29]:
len(df_2019_janfeb)

117055

In [24]:
df_2019["MONTH"].value_counts()

1     58746
2     58309
10    58058
8     58011
4     58002
11    57857
9     57794
12    57514
3     57491
6     57437
7     57253
5     57121
Name: MONTH, dtype: int64

In [11]:
df_2019.head()

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,LNKFW1MWT,PANLWT,Idx,counts
34206669,2019,1,1733.4595,20171000000401,10,5700,5700,8190,0.0,2511.8557,34206669,0
34206670,2019,1,2671.1416,20171000000402,10,4020,4000,8680,0.0,3872.8748,34206670,0
34206671,2019,1,2671.1416,20171000000403,10,4020,4000,8680,0.0,3872.8748,34206671,0
34206672,2019,1,2648.5806,20171000000404,10,4720,4720,5080,0.0,3837.9045,34206672,0
34206673,2019,1,1967.3134,20171000000601,10,3255,3130,9480,0.0,0.0000,34206673,0


In [12]:
df_2019["PANLWT_adj"] = df_2019["PANLWT"]/df_2019["counts"] 

In [42]:
count = 0
person_id_old = 0
occ_old = 0
ind_old = 0

transitions_occ = 0
transitions_ind = 0
transitions_both = 0
remained = 0
raw_transitions_occ = 0
raw_transitions_ind = 0 
raw_transitions_both = 0
raw_remained = 0

# get the transitions
for index, row in df_2019_janfeb.iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
#         print(occ_old, row["OCC2010"])
        # check if occupation changed
        if occ_old != row["OCC2010"]:
            raw_transitions_occ +=1 
            transitions_occ += 1 * row["PANLWT"]
            occ_old = row["OCC2010"]
            if ind_old != row["IND"]:
                raw_transitions_both += 1
                transitions_both += 1 * row["PANLWT"]
        else:
            remained += 1 * row["PANLWT"]
            raw_remained += 1
        if ind_old != row["IND"]: 
            raw_transitions_ind += 1
            transitions_ind += 1 * row["PANLWT"]

            
            
    else:
#         print("change")
#         print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
        ind_old = row["IND"]
    
    count +=1
    if count%1000000 == 0:
        print(count)

In [38]:
raw_transitions_occ

2968

In [43]:
(remained + transitions_occ)/1e6

155.54132286199902

In [46]:
transitions_occ/1e6

12.471811710500011

In [45]:
remained/1e6

143.06951115149903

In [13]:
count = 0
person_id_old = 0
occ_old = 0
ind_old = 0

transitions_occ = 0
transitions_ind = 0
transitions_both = 0
remained = 0
raw_transitions_occ = 0
raw_transitions_ind = 0 
raw_transitions_both = 0
raw_remained = 0

# get the transitions
for index, row in df_2019.iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
#         print(occ_old, row["OCC2010"])
        # check if occupation changed
        if occ_old != row["OCC2010"]:
            raw_transitions_occ +=1 
            transitions_occ += 1 * row["PANLWT_adj"]
            occ_old = row["OCC2010"]
            if ind_old != row["IND"]:
                raw_transitions_both += 1
                transitions_both += 1 * row["PANLWT_adj"]
        else:
            remained += 1 * row["PANLWT_adj"]
            raw_remained += 1
        if ind_old != row["IND"]: 
            raw_transitions_ind += 1
            transitions_ind += 1 * row["PANLWT_adj"]

            
            
    else:
#         print("change")
#         print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
        ind_old = row["IND"]
    
    count +=1
    if count%1000000 == 0:
        print(count)

In [17]:
remained/1e6

660.8965210472139

In [18]:
transitions_occ/1e6

67.20951433788343

In [16]:
transitions_occ/12

5600792.861490286

In [150]:
transitions_occ

61751947.47578335

In [151]:
remained 

480170554.2172433

In [137]:
df_2019["WTFINL_adj"]

34206669    1733.4595
34206670    2671.1416
34206671    2671.1416
34206672    2648.5806
34206673    1967.3134
              ...    
35810676     388.4827
35810779     402.8651
35810780     434.7944
35810781     420.2676
35810819     403.9888
Name: WTFINL_adj, Length: 693593, dtype: float64

In [128]:
np.mean(list(id_freq.values()))

3.047260249898951

In [117]:
f.loc[20191207029101][0]

1

In [101]:
f.index

Int64Index([20171000000401, 20171000000402, 20171000000403, 20171000000404,
            20171000000601, 20171000000602, 20171000005501, 20171000007401,
            20171000009501, 20171000010601,
            ...
            20191207014501, 20191207014602, 20191207015301, 20191207019201,
            20191207019202, 20191207019203, 20191207026701, 20191207026702,
            20191207026703, 20191207029101],
           dtype='int64', name='CPSIDP', length=227612)

In [118]:
d = {k:list(f.loc[k][0]) for k in f.index}

TypeError: 'numpy.int64' object is not iterable

In [114]:
d

{20171000000401: [1],
 20171000000402: [1],
 20171000000403: [1],
 20171000000404: [1],
 20171000000601: [1],
 20171000000602: [1],
 20171000005501: [1],
 20171000007401: [1],
 20171000009501: [1],
 20171000010601: [1],
 20171000011901: [1],
 20171000011902: [1],
 20171000014301: [1],
 20171000014302: [1],
 20171000014801: [1],
 20171000014803: [1],
 20171000020201: [1],
 20171000024001: [1],
 20171000024002: [1],
 20171000024003: [1],
 20171000024301: [1],
 20171000027801: [1],
 20171000027802: [1],
 20171000029401: [1],
 20171000029402: [1],
 20171000029403: [1],
 20171000029801: [1],
 20171000029901: [1],
 20171000029902: [1],
 20171000035202: [1],
 20171000035402: [1],
 20171000037101: [1],
 20171000037102: [1],
 20171000040001: [1],
 20171000040002: [1],
 20171000040801: [1],
 20171000040802: [1],
 20171000041301: [1],
 20171000043901: [1],
 20171000043902: [1],
 20171000044602: [1],
 20171000044801: [1],
 20171000044802: [1],
 20171000046702: [1],
 20171000047302: [1],
 201710000

In [ ]:
df_2019

In [94]:
df_2019.groupby('CPSIDP').size()

CPSIDP
20171000000401    1
20171000000402    1
20171000000403    1
20171000000404    1
20171000000601    1
                 ..
20191207019203    1
20191207026701    1
20191207026702    1
20191207026703    1
20191207029101    1
Length: 227612, dtype: int64

In [93]:
df_2019.iloc[67000:67025]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
34872722,2019,5,1374.6527,20180200479802,10,3050,3050,5070,34872722
34335287,2019,2,1382.7925,20180200483301,10,6200,6200,770,34335287
34454707,2019,3,1327.5720,20180200483301,10,6200,6200,770,34454707
34752911,2019,4,1320.2583,20180200483301,10,6200,6200,770,34752911
34872771,2019,5,1332.7810,20180200483301,10,6200,6200,770,34872771
34335288,2019,2,1701.3877,20180200483302,10,4110,4110,8680,34335288
34454708,2019,3,1592.9778,20180200483302,10,4110,4110,8680,34454708
34752912,2019,4,1582.3152,20180200483302,10,4110,4110,8680,34752912
34872772,2019,5,1444.6001,20180200483302,10,4110,4110,8680,34872772
34335292,2019,2,2014.8013,20180200483601,10,100,100,7870,34335292


In [91]:
df_2019.iloc[65000:65020]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
34562367,2019,3,3301.4499,20180107054802,10,430,430,770,34562367
34862294,2019,4,3381.3912,20180107054802,10,430,430,770,34862294
34324626,2019,1,4031.0320,20180107056101,10,5620,5620,4970,34324626
34444080,2019,2,3823.0513,20180107056101,10,5620,5620,4970,34444080
34562386,2019,3,3797.0694,20180107056101,10,5620,5620,4970,34562386
34862311,2019,4,4038.4385,20180107056101,10,5620,5620,4970,34862311
34324629,2019,1,3313.2506,20180107056104,10,4110,4110,8680,34324629
34444083,2019,2,3244.6364,20180107056104,10,630,620,8680,34444083
34562389,2019,3,3402.0618,20180107056104,10,4130,4130,8680,34562389
34862314,2019,4,3057.4363,20180107056104,10,4130,4130,8680,34862314


In [88]:
df_2019.iloc[10000:10020]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
34356478,2019,2,3701.7071,20171101695802,10,2830,2810,6672,34356478
34236206,2019,1,4269.3343,20171101696001,10,430,430,3990,34236206
34356486,2019,2,4233.9302,20171101696001,10,5860,5860,7680,34356486
34356510,2019,2,3384.8769,20171101697101,10,9140,9140,6190,34356510
34236247,2019,1,4899.1245,20171101697103,10,5140,5140,7280,34236247
34356512,2019,2,5162.4654,20171101697103,10,5140,5140,7280,34356512
34236248,2019,1,6556.3918,20171101697104,10,6355,6355,770,34236248
34356513,2019,2,6653.9252,20171101697104,10,8620,8620,670,34356513
34236255,2019,1,3797.5880,20171101697601,10,4700,4700,4970,34236255
34356516,2019,2,3806.2050,20171101697601,10,4700,4700,4970,34356516


In [37]:
df_2019_full = pd.read_pickle(path_data + "CPS_df_2019.pkl")
df_2019_full['Idx'] = df_2019_full.index

In [45]:
df_2019_full = df_2019_full[df_2019_full['WTFINL'].notna()]

In [46]:
df_2019_full = df_2019_full.sort_values(by = ['CPSIDP', 'Idx'])

In [47]:
df_2019_full = df_2019_full[(df_2019_full['OCC2010']!= 9999) & (df_2019_full['CPSIDP']!= 0)]

In [48]:
df_2019_full.head()

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,HFLAG,ASECWTH,PERNUM,WTFINL,...,ASECWT,AGE,RACE,HISPAN,EMPSTAT,LABFORCE,OCC,OCC2010,IND,Idx
34206669,2019,8,1,1733.4595,20171000000400,NaN,NaN,NaN,1,1733.4595,...,NaN,54,200,0,10,2,5700,5700,8190,34206669
34206670,2019,8,1,1733.4595,20171000000400,NaN,NaN,NaN,2,2671.1416,...,NaN,17,200,0,10,2,4020,4000,8680,34206670
34206671,2019,8,1,1733.4595,20171000000400,NaN,NaN,NaN,3,2671.1416,...,NaN,17,200,0,10,2,4020,4000,8680,34206671
34206672,2019,8,1,1733.4595,20171000000400,NaN,NaN,NaN,4,2648.5806,...,NaN,20,200,0,10,2,4720,4720,5080,34206672
34206673,2019,9,1,1967.3134,20171000000600,NaN,NaN,NaN,1,1967.3134,...,NaN,57,200,0,10,2,3255,3130,9480,34206673


In [50]:
df_2019_full["MISH"]

KeyError: 'MISH'

In [51]:
groups = df_2019_full.groupby("CPSID").agg({"WTFINL":["mean", "std"]})

In [56]:
groups[('WTFINL', 'mean')]

CPSID
20171000000400    2431.080825
20171000000600    2307.947000
20171000005500    2704.456300
20171000007400    2231.476750
20171000009500    2232.070100
                     ...     
20191207015300     266.611800
20191207018400     330.935200
20191207019200     389.420200
20191207026700     419.309033
20191207029100     403.988800
Name: (WTFINL, mean), Length: 143454, dtype: float64

In [54]:
groups.columns

MultiIndex([('WTFINL', 'mean'),
            ('WTFINL',  'std')],
           )

In [ ]:
df.groupby(['a'], as_index=False).agg(
                      {'c':['mean','std'],'b':'first', 'd':'first'})


In [32]:
df_2019_full["WTFINL"]

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,HFLAG,ASECWTH,PERNUM,WTFINL,CPSIDP,ASECWT,AGE,RACE,HISPAN,EMPSTAT,LABFORCE,OCC,OCC2010,IND
34206659,2019,1,1,1821.1212,20181200000100,NaN,NaN,NaN,1,1882.8985,20181200000101,NaN,68,200,0,36,1,0,9999,0
34206660,2019,1,1,1821.1212,20181200000100,NaN,NaN,NaN,2,1821.1212,20181200000102,NaN,64,200,0,10,2,7750,7750,3570
34206661,2019,3,1,1804.2193,20171200000300,NaN,NaN,NaN,1,2219.4429,20171200000301,NaN,64,200,0,36,1,0,9999,0
34206662,2019,3,1,1804.2193,20171200000300,NaN,NaN,NaN,2,1804.2193,20171200000302,NaN,59,200,0,10,2,5400,5400,7870
34206663,2019,5,1,3296.9061,20190100000500,NaN,NaN,NaN,1,3296.9061,20190100000501,NaN,31,200,0,10,2,8030,8030,3390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35810818,2019,70288,12,262.6019,20181007096200,NaN,NaN,NaN,1,262.6019,20181007096201,NaN,57,100,0,10,2,7020,7020,8090
35810819,2019,70291,12,403.9888,20191207029100,NaN,NaN,NaN,1,403.9888,20191207029101,NaN,39,100,100,10,2,2320,2320,7860
35810820,2019,70291,12,403.9888,20191207029100,NaN,NaN,NaN,2,482.3911,20191207029102,NaN,5,801,100,0,0,0,9999,0
35810821,2019,70291,12,403.9888,20191207029100,NaN,NaN,NaN,3,260.9972,20191207029103,NaN,10,100,200,0,0,0,9999,0


In [9]:
df_2019 = df[df["YEAR"] == 2019]

In [ ]:
df_2019 = df[df["YEAR"] == 2019]

In [24]:
count = 0
person_id_old = 0
occ_old = 0
ind_old = 0

transitions_occ = 0
transitions_ind = 0
transitions_both = 0
remained = 0
raw_transitions_occ = 0
raw_transitions_ind = 0 
raw_transitions_both = 0
raw_remained = 0

# get the transitions
for index, row in df_2019.iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
#         print(occ_old, row["OCC2010"])
        # check if occupation changed
        if occ_old != row["OCC2010"]:
            raw_transitions_occ +=1 
            transitions_occ += 1 * row["WTFINL"]
            occ_old = row["OCC2010"]
            if ind_old != row["IND"]:
                raw_transitions_both += 1
                transitions_both += 1 * row["WTFINL"]
        else:
            remained += 1 * row["WTFINL"]
            raw_remained += 1
        if ind_old != row["IND"]: 
            raw_transitions_ind += 1
            transitions_ind += 1 * row["WTFINL"]

            
            
    else:
#         print("change")
#         print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
        ind_old = row["IND"]
    
    count +=1
    if count%1000000 == 0:
        print(count)
#     if count > 100:
#         break

In [29]:
remained/1e6

1152.6292111585176

In [30]:
(remained + transitions_occ)/12

106997610.51725984

In [26]:
remained

1152629211.1585176

In [16]:
len(df_2019)

693593

In [17]:
raw_transitions_occ

45016

In [18]:
transitions_ind

151985405.57189944

In [28]:
transitions_occ/12

10945176.254050037

In [22]:
transitions_occ

131342115.04860044

In [19]:
transitions_both/transitions_ind

0.5318222065944523

In [20]:
transitions_both/transitions_occ

0.6154097163083688

In [14]:
transitions_both

80829213.76140033

In [10]:
df_2019.head()

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
34206669,2019,1,1733.4595,20171000000401,10,5700,5700,8190,34206669
34206670,2019,1,2671.1416,20171000000402,10,4020,4000,8680,34206670
34206671,2019,1,2671.1416,20171000000403,10,4020,4000,8680,34206671
34206672,2019,1,2648.5806,20171000000404,10,4720,4720,5080,34206672
34206673,2019,1,1967.3134,20171000000601,10,3255,3130,9480,34206673


In [20]:
df.iloc[100000:100010]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
301003,2000,3,1894.2097,19990302725901,12,274,4965,591,301003
556495,2000,4,1909.6596,19990302725901,10,274,4965,591,556495
625221,2000,5,1937.3754,19990302725901,10,274,4965,591,625221
747266,2000,6,1892.0428,19990302725901,10,274,4965,591,747266
301034,2000,3,2114.3566,19990302727301,10,17,310,641,301034
556528,2000,4,2046.9970,19990302727301,10,17,310,641,556528
724532,2000,5,2062.4065,19990302727301,10,17,310,641,724532
812342,2000,6,2099.2463,19990302727301,10,17,310,641,812342
556529,2000,4,2158.8727,19990302727401,10,276,4720,682,556529
657903,2000,5,2338.2644,19990302727401,10,276,4720,682,657903


In [28]:
df[102730:102750]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
567634,2000,4,1111.4283,19990303273501,10,53,1360,882,567634
625243,2000,5,1120.3141,19990303273501,10,53,1360,882,625243
311930,2000,3,1102.3858,19990303273502,10,243,4700,541,311930
567635,2000,4,1116.1194,19990303273502,10,243,4700,541,567635
625244,2000,5,1134.9153,19990303273502,10,243,4700,541,625244
311961,2000,3,1102.3858,19990303274901,10,337,5120,700,311961
567663,2000,4,1116.1194,19990303274901,10,389,5940,700,567663
671981,2000,5,1134.9153,19990303274901,10,376,5330,700,671981
778215,2000,6,1185.0610,19990303274901,10,376,5330,700,778215
311962,2000,3,1091.2312,19990303274902,10,243,4700,620,311962


In [31]:
count = 0
person_id_old = 0
occ_old = 0
for index, row in df[102730:102750].iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
        print(occ_old, row["OCC2010"])
        occ_old = row["OCC2010"]
        pass
    else:
        print("change")
        print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
    
    count +=1
    if count > 100:
        break

change
id 19990303273501.0
1360.0 1360.0
change
id 19990303273502.0
4700.0 4700.0
4700.0 4700.0
change
id 19990303274901.0
5120.0 5940.0
5940.0 5330.0
5330.0 5330.0
change
id 19990303274902.0
4700.0 4700.0
4700.0 4700.0
4700.0 4700.0
change
id 19990303275302.0
710.0 710.0
710.0 710.0
710.0 710.0
change
id 19990303277201.0
800.0 430.0
430.0 430.0


In [34]:
count = 0
person_id_old = 0
occ_old = 0
transitions = 0
raw_transitions = 0
for index, row in df[102730:102750].iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
        print(occ_old, row["OCC2010"])
        if occ_old != row["OCC2010"]:
            raw_transitions +=1 
            transitions += 1 *  row["WTFINL"]
            occ_old = row["OCC2010"]
        pass
    else:
        print("change")
        print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
    
    count +=1
    if count > 100:
        break

change
id 19990303273501.0
1360.0 1360.0
change
id 19990303273502.0
4700.0 4700.0
4700.0 4700.0
change
id 19990303274901.0
5120.0 5940.0
5940.0 5330.0
5330.0 5330.0
change
id 19990303274902.0
4700.0 4700.0
4700.0 4700.0
4700.0 4700.0
change
id 19990303275302.0
710.0 710.0
710.0 710.0
710.0 710.0
change
id 19990303277201.0
800.0 430.0
430.0 430.0


In [35]:
raw_transitions

3

In [36]:
transitions

3247.2467

In [39]:
len(df)

16869617

In [40]:
count = 0
person_id_old = 0
occ_old = 0
ind_old = 0

transitions_occ = 0
transitions_ind = 0
transitions_both = 0
raw_transitions_occ = 0
raw_transitions_ind = 0 
raw_transitions_both = 0
for index, row in df.iterrows():
    person_id_new = row["CPSIDP"]
    
    if person_id_new == person_id_old:
        #check for change in occupation
#         print(occ_old, row["OCC2010"])
        if occ_old != row["OCC2010"]:
            raw_transitions +=1 
            transitions += 1 *  row["WTFINL"]
            occ_old = row["OCC2010"]
        if ind_old != row["IND"]
            
        pass
    else:
#         print("change")
#         print("id", person_id_new)
        person_id_old = person_id_new
        occ_old = row["OCC2010"]
    
    count +=1
    if count%1000000 == 0:
        print(count)
#     if count > 100:
#         break

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [41]:
raw_transitions

1861709

In [42]:
transitions

4522386345.174492

In [43]:
transitions/20

226119317.2587246

In [6]:
df.loc[df["CPSIDP"] == 19990303144203]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
309268,2000,3,475.0559,19990303144203,10,467,2540,862,309268
466089,2000,3,NaN,19990303144203,10,467,2540,862,466089


In [21]:
df = df[df['WTFINL'].notna()]

In [ ]:
19990303144203

In [ ]:
19990303152101 	

In [27]:
df[102730:102750]

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND,Idx
567634,2000,4,1111.4283,19990303273501,10,53,1360,882,567634
625243,2000,5,1120.3141,19990303273501,10,53,1360,882,625243
311930,2000,3,1102.3858,19990303273502,10,243,4700,541,311930
567635,2000,4,1116.1194,19990303273502,10,243,4700,541,567635
625244,2000,5,1134.9153,19990303273502,10,243,4700,541,625244
311961,2000,3,1102.3858,19990303274901,10,337,5120,700,311961
567663,2000,4,1116.1194,19990303274901,10,389,5940,700,567663
671981,2000,5,1134.9153,19990303274901,10,376,5330,700,671981
778215,2000,6,1185.0610,19990303274901,10,376,5330,700,778215
311962,2000,3,1091.2312,19990303274902,10,243,4700,620,311962


In [22]:
len(df)

16895274

In [8]:
df_sort= df.sort_values("CPSIDP")

ValueError: No axis named CPSIDP for object type DataFrame

In [7]:
df_grouped.head()

,YEAR,MONTH,WTFINL,CPSIDP,EMPSTAT,OCC,OCC2010,IND
15,2000,1,233.3782,19981204103001,12,869,6260,60
20,2000,1,233.4906,19981204102801,12,783,8140,41
28,2000,1,290.0727,19981103262701,10,783,8140,41
34,2000,1,3309.0053,19981202764601,21,558,6200,60
36,2000,1,3285.7963,19981204464201,10,783,8140,282
...,...,...,...,...,...,...,...,...
38481417,2021,10,373.8029,20201006877501,10,3740,3740,9590
38481429,2021,10,579.4686,20210906771501,10,4220,4220,7860
38481431,2021,10,347.3348,20210806766401,10,2770,2760,9090
38481432,2021,10,381.3091,20210806766402,12,9645,5620,5391


In [ ]:
ipums_occ_set = set(occ10_list) # create a set of the ipums occupations
transitions = np.zeros((len(occ10_list ),len(occ10_list )))# where we will store the raw number of transitions
#progress check
for ii,i in enumerate(sampled_twice):
    if ii == round(len(sampled_twice)*0.5):
        print ('50% done')
    if ii == round(len(sampled_twice)*0.75):
        print ('70% done')    
    if ii == round(len(sampled_twice)*0.95):
        print ('95% done')
        
    p_data = new_df[new_df['CPSIDP']==i] # person specific data
    career_path = list(p_data['OCC']) # the occupations that person was in over the sampled periods (ordered chronologically)
    pweight = list(p_data['WTFINL'])
    
        # if that person had a job change in their career path:
    if career_path.count(career_path[0]) != len(career_path):
        # figure out the transitions
        for i, occ in enumerate(career_path):
            if i < len(career_path)-1 and career_path[i] !=career_path[i+1]:
                old_occ = career_path[i]
                new_occ = career_path[i+1]
                if {old_occ,new_occ} <= ipums_occ_set: # if the CPS occupations are in our ipums occupation set (not all of them are... )
                    # put person (weighted by person weight) transitions in transition matrix         
                     transitions[occ10_list.index(old_occ),occ10_list.index(new_occ)] += pweight[i+1]
